### Note!!!

This notebook requires atleast 50GB RAM and 16GB VRAM to run, ensure to have colab pro

In [1]:
!pip install -q PymuPDF
!pip install -q python-dotenv
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.0 MB/s eta 0:00:00


In [ ]:
import os
import fitz
import numpy as np
import json
import re

from openai import OpenAI
from tqdm import tqdm

import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, pipeline, AutoModelForCausalLM
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def gen_text_chunk(pdf_path, n, overlap):
  pdf = fitz.open(pdf_path)
  text = ""
  for page in pdf:
    text += page.get_text("text")

  chunk = [text[i: i+n] for i in range(0, len(text), n-overlap)]
  return chunk

text_chunks = gen_text_chunk("data/AI_Information.pdf", n=1000, overlap=200)
text_chunks[:5]

['Understanding Artificial Intelligence \nChapter 1: Introduction to Artificial Intelligence \nArtificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot \nto perform tasks commonly associated with intelligent beings. The term is frequently applied to \nthe project of developing systems endowed with the intellectual processes characteristic of \nhumans, such as the ability to reason, discover meaning, generalize, or learn from past \nexperience. Over the past few decades, advancements in computing power and data availability \nhave significantly accelerated the development and deployment of AI. \nHistorical Context \nThe idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. \nHowever, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop \nin 1956 is widely considered the birthplace of AI. Early AI research focused on problem-solving \nand symbolic methods. The 1980s saw

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
gen_tokenizer = AutoTokenizer.from_pretrained("unsloth/Meta-Llama-3.1-8B-Instruct")
gen_model = AutoModelForCausalLM.from_pretrained("unsloth/Meta-Llama-3.1-8B-Instruct",
    torch_dtype="auto",
    device_map="auto")


embed_model = AutoModel.from_pretrained("BAAI/bge-base-en").to(device)
embed_tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-base-en")

In [25]:
def generate_questions(chunk, num_questions=5, model="unsloth/Meta-Llama-3.1-8B-Instruct"):
    """
    Generates relevant questions that can be answered from the given text chunk.

    Args:
    text_chunk (str): The text chunk to generate questions from.
    num_questions (int): Number of questions to generate.
    model (str): The model to use for question generation.

    Returns:
    List[str]: List of generated questions.
    """
    # Define the system prompt to guide the AI's behavior
    system_prompt = "You are an expert at generating relevant questions from text. Create concise questions that can be answered using only the provided text. Focus on key information and concepts."

    # Define the user prompt with the text chunk and the number of questions to generate
    user_prompt = f"""
    Based on the following text, generate {num_questions} different questions that can be answered using only this text:

    {chunk}

    Format your response as a numbered list of questions ONLY, with no additional text.
    """

    # Generate a response from the AI model based on the system prompt and text chunk
    text = gen_tokenizer.apply_chat_template(
        conversation=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = gen_tokenizer([text], return_tensors="pt").to(device)

    generated_ids = gen_model.generate(
        **model_inputs,
        # max_new_tokens=50,
        # temperature=0,
        do_sample=True
    )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = gen_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    print("===========================================")
    print(f"questions: \n{response}")
    print("===========================================")
    questions = []

    # Extract questions using regex pattern matching
    for line in response.split('\n'):
        # Remove numbering and clean up whitespace
        cleaned_line = re.sub(r'^\d+\.\s*', '', line.strip())
        if cleaned_line and cleaned_line.endswith('?'):
            questions.append(cleaned_line)

    return questions


In [27]:
generate_questions(text_chunks[2], 3)

questions: 
1. What is the primary focus of machine learning in artificial intelligence?
2. What type of data is used to train supervised learning algorithms?
3. What is an example of a common technique used in unsupervised learning?


['What is the primary focus of machine learning in artificial intelligence?',
 'What type of data is used to train supervised learning algorithms?',
 'What is an example of a common technique used in unsupervised learning?']

In [ ]:
def create_embeddings(text):
    """
    Create embeddings for text using the loaded embedding model.

    Args:
        text: str or list of str - input text(s) to embed

    Returns:
        list[np.ndarray]: list of normalized embeddings, each of shape (dim,)
    """
    # Handle single string input
    is_single = isinstance(text, str)
    if is_single: text = [text]

    # Tokenize with error handling
    try:
        inputs = tokenizer(
            text,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        ).to(device)
    except Exception as e:
        print(f"Tokenization error: {e}")
        return None

    # Generate embeddings with no gradient computation
    try:
        with torch.no_grad():
            output = model(**inputs)
            # Use CLS token embedding [CLS] at position 0
            cls_emb = output.last_hidden_state[:, 0, :]
            # L2 normalize for cosine similarity
            emb_normalized = F.normalize(cls_emb, p=2, dim=1)

        # Convert to list of np.ndarray, each of shape (dim,)
        embeddings = [emb.cpu().numpy() for emb in emb_normalized]

        # Return single embedding if input was single string
        return embeddings

    except Exception as e:
        print(f"Embedding generation error: {e}")
        return None

In [ ]:
class SimpleVectorStore:
    """
    A simple vector store implementation using NumPy.
    """
    def __init__(self):
        """
        Initialize the vector store.
        """
        self.vectors = []
        self.texts = []
        self.metadata = []

    def add_item(self, text, embedding, metadata=None):
        """
        Add an item to the vector store.

        Args:
        text (str): The original text.
        embedding (List[float]): The embedding vector.
        metadata (dict, optional): Additional metadata.
        """
        self.vectors.append(embedding)
        self.texts.append(text)
        self.metadata.append(metadata or {})

    def similarity_search(self, query_embedding, k=5):
        """
        Find the most similar items to a query embedding.

        Args:
        query_embedding (List[float]): Query embedding vector.
        k (int): Number of results to return.

        Returns:
        List[Dict]: Top k most similar items with their texts and metadata.
        """
        if not self.vectors:
            return []

        # Convert query embedding to numpy array
        query_vector = query_embedding

        # Calculate similarities using cosine similarity
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = cosine_similarity(query_vector, vector)
            similarities.append((i, similarity))

        # Sort by similarity (descending)
        similarities.sort(key=lambda x: x[1], reverse=True)

        # Return top k results
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": score
            })

        return results


In [ ]:
vector_store = SimpleVectorStore()

print("Processing chunks and generating questions...")
for i, chunk in enumerate(tqdm(text_chunks, desc="Processing Chunks")):
    # Create embedding for the chunk itself
    chunk_embedding = create_embeddings(chunk)[0].reshape(1, -1)

    # Add the chunk to the vector store
    vector_store.add_item(
        text=chunk,
        embedding=chunk_embedding,
        metadata={"type": "chunk", "index": i}
    )

    # Generate questions for this chunk
    questions = generate_questions(chunk, num_questions=5)

    # Create embeddings for each question and add to vector store
    for j, question in enumerate(questions):
        question_embedding = create_embeddings(question)[0].reshape(1, -1)

        # Add the question to the vector store
        vector_store.add_item(
            text=question,
            embedding=question_embedding,
            metadata={"type": "question", "chunk_index": i, "original_chunk": chunk}
        )


Processing chunks and generating questions...


Processing Chunks:   2%|▏         | 1/42 [00:03<02:31,  3.69s/it]

questions: 
1. What is the primary ability of a digital computer or computer-controlled robot in the context of AI?
2. What intellectual processes are characteristic of humans that AI systems aim to replicate?
3. What event in 1956 is widely considered the birthplace of AI?
4. What area of focus did early AI research primarily concentrate on?
5. What period saw a rise in the use of new methods in AI research?


Processing Chunks:   5%|▍         | 2/42 [00:07<02:25,  3.63s/it]

questions: 
1. When is the Dartmouth Workshop considered the birthplace of AI?
2. What type of methods were early AI research focused on?
3. What advancements were seen in the 1990s and 2000s in AI?
4. What has recent breakthroughs in deep learning revolutionized in the field of AI?
5. What are some examples of AI systems that are increasingly prevalent in everyday life?


...

questions: 
1. What are the key components of building trust in AI?
2. What is the purpose of public engagement and education in AI?
3. What are the essential elements for advancing AI capabilities?
4. What is the goal of responsible development and deployment of AI?
5. What is necessary for addressing the challenges of AI?


Processing Chunks:  98%|█████████▊| 41/42 [02:07<00:03,  3.02s/it]

questions: 
1. What are the key principles involved in responsible AI development and deployment?
2. Why is global collaboration and cooperation important for addressing AI-related challenges?
3. What is required to prepare the workforce for the future of AI?
4. What type of education is promoted to equip individuals with AI-related skills?
5. What approach focuses on developing AI systems with human values in mind?


Processing Chunks: 100%|██████████| 42/42 [02:10<00:00,  3.11s/it]

questions: 
1. What is a key goal of a human-centered approach to AI?
2. What are the primary considerations when developing AI systems using a human-centered approach?
3. What are the benefits of embracing a human-centered approach to AI development?
4. What is the primary focus of a human-centered approach to AI development?
5. What is required for the path forward in AI development and deployment?


In [ ]:
def semantic_search(query, vector_store, k=5):
    """
    Performs semantic search using the query and vector store.

    Args:
    query (str): The search query.
    vector_store (SimpleVectorStore): The vector store to search in.
    k (int): Number of results to return.

    Returns:
    List[Dict]: Top k most relevant items.
    """
    # Create embedding for the query
    query_embedding = create_embeddings(query)[0].reshape(1, -1)

    # Search the vector store
    results = vector_store.similarity_search(query_embedding, k=k)

    return results


In [ ]:
# Load the validation data from a JSON file
with open('data/val.json') as f:
    data = json.load(f)

# Extract the first query from the validation data
query = data[0]['question']

# Perform semantic search to find relevant content
search_results = semantic_search(query, vector_store, k=5)

print("Query:", query)
print("\nSearch Results:")

# Organize results by type
chunk_results = []
question_results = []

for result in search_results:
    if result["metadata"]["type"] == "chunk":
        chunk_results.append(result)
    else:
        question_results.append(result)

# Print chunk results first
print("\nRelevant Document Chunks:")
for i, result in enumerate(chunk_results):
    print(f"Context {i + 1} (similarity: {result['similarity']:.4f}):")
    print(result["text"][:300] + "...")
    print("=====================================")

# Then print question matches
print("\nMatched Questions:")
for i, result in enumerate(question_results):
    print(f"Question {i + 1} (similarity: {result['similarity']:.4f}):")
    print(result["text"])
    chunk_idx = result["metadata"]["chunk_index"]
    print(f"From chunk {chunk_idx}")
    print("=====================================")

Query: What is 'Explainable AI' and why is it considered important?

Search Results:

Relevant Document Chunks:

Matched Questions:
Question 1 (similarity: 0.9364):
What is the purpose of Explainable AI (XAI) techniques?
From chunk 36
Question 2 (similarity: 0.9232):
What is the primary goal of Explainable AI (XAI)?
From chunk 10
Question 3 (similarity: 0.9223):
What is the main goal of Explainable AI (XAI) techniques?
From chunk 37
Question 4 (similarity: 0.9209):
What is the primary goal of transparency and explainability in AI systems?
From chunk 36
Question 5 (similarity: 0.9116):
What are the goals of research in Explainable AI (XAI)?
From chunk 29


In [47]:
def prepare_context(search_results):
    """
    Prepares a unified context from search results for response generation.

    Args:
    search_results (List[Dict]): Results from semantic search.

    Returns:
    str: Combined context string.
    """
    # Extract unique chunks referenced in the results
    chunk_indices = set()
    context_chunks = []

    # First add direct chunk matches
    for result in search_results:
        if result["metadata"]["type"] == "chunk":
            chunk_indices.add(result["metadata"]["index"])
            context_chunks.append(f"Chunk {result['metadata']['index']}:\n{result['text']}")

    # Then add chunks referenced by questions
    for result in search_results:
        if result["metadata"]["type"] == "question":
            chunk_idx = result["metadata"]["chunk_index"]
            if chunk_idx not in chunk_indices:
                chunk_indices.add(chunk_idx)
                context_chunks.append(f"Chunk {chunk_idx} (referenced by question '{result['text']}'):\n{result['metadata']['original_chunk']}")

    # Combine all context chunks
    full_context = "\n\n".join(context_chunks)
    return full_context


In [ ]:
# use api
from dotenv  import load_dotenv
load_dotenv("conf.env")

client = OpenAI(
    base_url = os.getenv("GEMINI_BASE_URL"),
    api_key = os.getenv("GEMINI_API_KEY")
)

In [49]:
def generate_response(query, context, model="gemini-2.5-flash"):
    """
    Generates a response based on the query and context.

    Args:
    query (str): User's question.
    context (str): Context information retrieved from the vector store.
    model (str): Model to use for response generation.

    Returns:
    str: Generated response.
    """
    system_prompt = "You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly from the provided context, respond with: 'I do not have enough information to answer that.'"

    user_prompt = f"""
        Context:
        {context}

        Question: {query}

        Please answer the question based only on the context provided above. Be concise and accurate.
    """

    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    return response.choices[0].message.content


In [50]:
# Prepare context from search results
context = prepare_context(search_results)

# Generate response
response_text = generate_response(query, context)

print("\nQuery:", query)
print("\nResponse:")
print(response_text)


Query: What is 'Explainable AI' and why is it considered important?

Response:
Explainable AI (XAI) aims to make AI systems and their decisions more transparent and understandable, providing insights into how AI models make decisions. It is considered important for building trust in AI systems, enabling users to assess their fairness, accuracy, and reliability, and enhancing accountability.


In [53]:
def evaluate_response(query, response, reference_answer, model="gemini-2.5-flash"):
    """
    Evaluates the AI response against a reference answer.

    Args:
    query (str): The user's question.
    response (str): The AI-generated response.
    reference_answer (str): The reference/ideal answer.
    model (str): Model to use for evaluation.

    Returns:
    str: Evaluation feedback.
    """
    # Define the system prompt for the evaluation system
    evaluate_system_prompt = """You are an intelligent evaluation system tasked with assessing AI responses.

        Compare the AI assistant's response to the true/reference answer, and evaluate based on:
        1. Factual correctness - Does the response contain accurate information?
        2. Completeness - Does it cover all important aspects from the reference?
        3. Relevance - Does it directly address the question?

        Assign a score from 0 to 1:
        - 1.0: Perfect match in content and meaning
        - 0.8: Very good, with minor omissions/differences
        - 0.6: Good, covers main points but misses some details
        - 0.4: Partial answer with significant omissions
        - 0.2: Minimal relevant information
        - 0.0: Incorrect or irrelevant

        Provide your score with justification.
    """

    # Create the evaluation prompt
    evaluation_prompt = f"""
        User Query: {query}

        AI Response:
        {response}

        Reference Answer:
        {reference_answer}

        Please evaluate the AI response against the reference answer.
    """

    # Generate evaluation
    eval_response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": evaluate_system_prompt},
            {"role": "user", "content": evaluation_prompt}
        ]
    )

    return eval_response.choices[0].message.content


In [54]:
# Get reference answer from validation data
reference_answer = data[0]['ideal_answer']

# Evaluate the response
evaluation = evaluate_response(query, response_text, reference_answer)

print("\nEvaluation:")
print(evaluation)


Evaluation:
Score: 1.0

Justification: The AI response perfectly captures the definition of Explainable AI (XAI) and accurately lists the key reasons for its importance (transparency, understanding, trust, fairness, accountability), aligning almost identically with the reference answer. The slight expansion on "enabling users to assess their fairness, accuracy, and reliability" is a valid and relevant elaboration that enhances the answer without introducing any inaccuracies or deviating from the core message of the reference.


In [ ]:
ai_res = create_embeddings(ai_response_text)[0].reshape(1, -1)
ideal_res = create_embeddings(data[0]['ideal_answer'])[0].reshape(1, -1)

print(cosine_similarity(ai_res, ideal_res))

[[0.99685514]]
